In [1]:
## Setup
from __future__ import print_function
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import csv


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def get_session():
    """Create a session that dynamically allocates memory."""
    # See: https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


In [2]:
import utilities

### Global variables ###
TRAIN_RATIO = 0.6    # Fraction of data to use for training
#VAL_RATIO   = 0.2    # Fraction of data to use for validation
N_LEVELS    = 20
BATCH_SIZE = 64


In [3]:
# Load the data from csv files:
tmp, x_data, y_norm = utilities.load_data()
print(x_data.dtype)

# Preprocess for squeezenet


N_training = int(TRAIN_RATIO*x_data.shape[0])
N_validation = x_data.shape[0]- N_training

y_data, mean_vals = utilities.discretize_outputs(y_norm[0:N_training], N_LEVELS)
y_data, tmp = utilities.discretize_outputs(y_norm, N_LEVELS)

x_train = x_data[0:N_training,...]
y_train = y_data[0:N_training]

x_val = x_data[N_training:,...]
y_val = y_data[N_training:]

print(x_train.shape)
batch_x, batch_y = utilities.sample_3x(BATCH_SIZE, x_train, y_train)
print(batch_x.shape)


Loading vector data
Loading image data
Preprocessing data
float32
(5640, 192, 192)
(64, 192, 192, 3)


In [7]:
from classifiers.squeezenet import SqueezeNet

tf.reset_default_graph() # remove all existing variables in the graph 
sess = get_session() # start a new Session
# Load pretrained SqueezeNet model
SAVE_PATH = 'classifiers/squeezenet.ckpt'
#if not os.path.exists(SAVE_PATH):
#    raise ValueError("You need to download SqueezeNet!")

model = SqueezeNet(N_LEVELS, sess=sess)
#model = SqueezeNet(N_LEVELS,save_path=SAVE_PATH, sess=sess)


uninitialized_vars = []
for var in tf.global_variables():
    try:
        sess.run(var)
    except tf.errors.FailedPreconditionError:
        uninitialized_vars.append(var)
        
init = tf.variables_initializer(uninitialized_vars)
sess.run(init)
#print(uninitialized_vars)



In [ ]:
# Now train!

disp_period = 1
save_period = 1000
training_iters = 40000


print('initializing...')
lr = 5e-3
lr_decay = 0.99
with tf.device('/cpu:0'):
        # Initializing the variables

    train_history = []
    val_history = []
    naive_history = []
    
    # Launch the graph
    step = 1
    ind_train = []
    train_inds = []
    val_inds = []

    smoothed_train = []
    smooth_val = 1-0.999

    print('Training')
    # Keep training until reach max iterations
    while step  < training_iters:
        if step % 100 is 0:
            lr *= lr_decay
        if step % 2000 is 0:
            lr *= 0.1
        lr = np.max([lr, 0.00000001])
        # these batches are of size 1?
        # for now, just grab a random sequence of data:
        batch_x, batch_y = utilities.sample_3x(BATCH_SIZE, x_train, y_train)

        # Run optimization op (backprop)
        o,mse = sess.run([model.optimizer,model.loss], feed_dict={model.image: batch_x, model.labels: batch_y, model.lr:[lr]})
        train_inds.append(step)
        if step > 1:
            train_history.append(mse*smooth_val + (1-smooth_val)*train_history[-1])
        else:
            train_history.append(mse)
            
        step += 1
        if step % disp_period is 0:
            # Calculate val accuracy:
            N_smooth = 10;
            batch_x, batch_y = utilities.sample_3x(BATCH_SIZE, x_val, y_val)
            vmse = sess.run(model.loss, feed_dict={model.image: batch_x, model.labels: batch_y})
            val_history.append(vmse/N_smooth)
            val_inds.append(step)
            
            for smooth_num in range(N_smooth-1):
                batch_x, batch_y = utilities.sample_3x(BATCH_SIZE, x_val, y_val)
                vmse = sess.run(model.loss, feed_dict={model.image: batch_x, model.labels: batch_y})
                val_history[-1]+=(vmse/N_smooth)

            
            print('Step: ', step, ' MSE: ', train_history[-1], ' Val: ', val_history[-1], 'LR: ', lr)
            plt.semilogy(train_inds, train_history,alpha=0.9)
            plt.semilogy(val_inds, val_history,alpha=0.6)
            plt.semilogy([0,step],[0.0069,0.0069],':')
            plt.legend(['Training','Validation','Constant est.'])
            plt.xlabel('Epoch')
            plt.ylabel('RMSE')
            plt.savefig('pretrained_net_perf.png')
            
        if step % save_period is 0:
            print('Computing sequence prediction - this may take a while. ')
            y_pred = np.zeros_like(y_data)

            for k in range(0,x_data.shape[0]-(x_data.shape[0]%BATCH_SIZE),BATCH_SIZE):
                if k % 1024 is 0:
                    print('k = ', k)
                bx,by = utilities.sample_3x(BATCH_SIZE,x_data,y_data,k)
                pred = sess.run(model.prediction, feed_dict={model.image: bx})
                y_pred[k:k+BATCH_SIZE] = pred
            np.save('pretrained_cnn_pred',y_pred)





In [ ]:
# Reconstruct signal using a randomwalk model:
print('Computing sequence prediction - this may take a while. ')
y_pred = np.zeros_like(y_data)

for k in range(0,x_data.shape[0]-(x_data.shape[0]%BATCH_SIZE),BATCH_SIZE):
    if k % 1024 is 0:
        print('k = ', k)
    bx,by = utilities.sample_3x(BATCH_SIZE,x_data,y_data,k)
    pred = sess.run(model.prediction, feed_dict={model.image: bx})
    y_pred[k:k+BATCH_SIZE] = pred
    


In [ ]:



# Blue background corresponds to data used during training
def plot_data(ns, ne, save=False):    

    print('plotting from ', ns, ' to ', ne)
    
    print('High correlation (near 1) means we are predicting well.')
    m1 = y_norm[ns:ne].mean()
    c1 = np.sqrt(np.correlate(y_norm[ns:ne]-m1,y_norm[ns:ne]-m1)[0])
    m2 = r[ns:ne].mean()
    c2 = np.sqrt(np.correlate(r[ns:ne]-m2,r[ns:ne]-m2)[0])
    c12 = (np.correlate(y_norm[ns:ne]-m1,r[ns:ne]-m2))[0]/(c1*c2)
    print('Correlation between error and signal:', (np.abs(c12)))
    
    plt.subplot(2,1,1)
    plt.ylabel('Error Rate')
#    plt.axvspan(0,0.6*y_norm.shape[0],facecolor='b',alpha=0.1)
#    plt.axvspan(0.6*y_norm.shape[0],y_norm.shape[0],facecolor='g',alpha=0.1)
    plt.plot(y_norm[ns:ne],'.',alpha=0.6)
    plt.plot(r[ns:ne],'-.',alpha=0.6)
#    plt.xlim([ns,ne])
    plt.legend(['Actual', 'Predicted'])
    plt.plot([0,ne-ns],[0,0],':')

    plt.subplot(2,1,2)
    plt.axvspan(0,0.6*y_norm.shape[0],facecolor='b',alpha=0.1)
    plt.axvspan(0.6*y_norm.shape[0],y_norm.shape[0],facecolor='g',alpha=0.1)
    plt.ylabel('Cumulative Error')
    plt.plot(np.cumsum(r[ns:ne]))
    plt.plot(np.cumsum(y_norm[ns:ne]))
    plt.plot([0,ne-ns],[0,0],':')
    plt.legend(['Predicted','Actual'])
    
    if(save):
        plt.savefig('pred.png',dpi=720)
    else:
        plt.show()
    return plt

def plot_error(ns, ne):
    print('High correlation (near 1) means we are predicting noise.')
    m1 = y_norm[ns:ne].mean()
    c1 = np.sqrt(np.correlate(y_norm[ns:ne]-m1,y_norm[ns:ne]-m1)[0])
    m2 = np.mean(r[ns:ne]-y_norm[ns:ne])
    c2 = np.sqrt(np.correlate(r[ns:ne]-m2-y_norm[ns:ne],r[ns:ne]-m2-y_norm[ns:ne])[0])
    c12 = (np.correlate(y_norm[ns:ne]-m1,r[ns:ne]-y_norm[ns:ne]-m2))[0]/(c1*c2)
    print('Correlation between error and signal:', (np.abs(c12)))
    
    plt.ylabel('Prediction Errors')
    plt.plot(r[ns:ne]-y_norm[ns:ne])
    plt.plot(-y_norm[ns:ne],':')
    plt.legend(['Error', 'Signal'])
    plt.show()



# Can load a previously computed prediction - useful if cold starting a notebook.
r = mean_vals[(y_pred[:]).astype(np.int32)];
np.save('scnn_predictions',r)

print('RMSE: ',utilities.calc_rmse(r, y_norm))

# smooth the reconstructed signal:
#r = np.zeros_like(reconstructed_signal)
#alpha = 1.0
#for k in range(reconstructed_signal.shape[0]):
#    if(k==0):
#        r[k] = reconstructed_signal[0]
#    else:
#        r[k] = alpha*reconstructed_signal[k] + (1-alpha)*r[k-1]
        
plt.plot(y_norm)
plt.plot(training_data_indices, r[training_data_indices],'.')
plt.plot(validation_data_indices, r[validation_data_indices],'.')
plt.legend(['Actual','Train','Val'])
plt.show()

#plt.title('Learning Curve')
#naive_val = 0
#for k in range(y_data.shape[0]):
#    naive_val += calc_rmse(y_data[k,:],np.zeros(OUTPUT_DIM))
#naive_val /= y_data.shape[0]

#plt.plot(train_history,'.',alpha=0.3)
#plt.plot(val_history,'.',alpha=0.3)
#plt.plot(naive_val*np.ones_like(train_history),':')
#plt.legend(['Training','Validation','Naive'])
#plt.xlabel('Epoch')
#plt.ylabel('RMSE')
#plt.show()


plot_data(0,y_data.shape[0])
plot_error(300,500)


In [ ]:
# Compute discretization and check its error:
y_d,binvals = utilities.discretize_outputs(err_norm[train_inds],N_LEVELS)
y_d,bv = utilities.discretize_outputs(err_norm,N_LEVELS)


print('RMSE from discretziation', utilities.calc_rmse(binvals[(y_d[:]).astype(np.int32)], err_norm))
print('RMSE from naive mean guess', utilities.calc_rmse(err_norm, np.mean(err_norm)*np.ones_like(err_norm)))

plt.subplot(1,2,1)
plt.plot(err_norm)
plt.plot(binvals[(y_d[:]).astype(np.int32)],alpha=0.3);
plt.subplot(1,2,2)
plt.plot(binvals,'.')
plt.plot(bv,'.')

# Discretized value:
y_norm_disc = binvals[(y_d[:]).astype(np.int32)];

In [ ]:
# Play around with the SQSS and see if we can get it to work:

data_dict = {}

N_steps = 10

#data_dict['imu'] = tf.placeholder('float',shape=[N_steps,None,None,2],name='input_image')

data_dict['x_acc'] = imu_data[:,0]
data_dict['y_acc'] = imu_data[:,1]
data_dict['z_acc'] = imu_data[:,2]
data_dict['x_gyr'] = imu_data[:,3]
data_dict['y_gyr'] = imu_data[:,4]
data_dict['z_gyr'] = imu_data[:,5]
data_dict['flows'] = imu_data[:,6]
data_keys = data_dict.keys()

data_cntxt['x_acc'] = (imu_data[:,0].mean()
data_cntxt['y_acc'] = (imu_data[:,1].mean()
data_cntxt['z_acc'] = (imu_data[:,2].mean()
data_cntxt['x_gyr'] = (imu_data[:,3].mean()
data_cntxt['y_gyr'] = (imu_data[:,4].mean()
data_cntxt['z_gyr'] = (imu_data[:,5].mean()
data_cntxt['flows'] = (imu_data[:,6]).mean()

batch_size = 32
num_unroll = 20
num_enqueue_threads = 4
lstm_size = 64
state_size = 64
cell = tf.contrib.rnn.BasicLSTMCell(num_units=lstm_size)

initial_state_values = tf.zeros((state_size,), dtype=tf.float32)
initial_states = {"lstm_state": initial_state_values}

batch = tf.batch_sequences_with_states(
    input_key=data_dict.keys(),
    input_sequences=data_dict,
    input_context=data_cntxt,
    input_length=tf.shape(data["x_acc"])[0],
    initial_states=initial_states,
    num_unroll=num_unroll,
    batch_size=batch_size,
    num_threads=num_enqueue_threads,
    capacity=batch_size * num_enqueue_threads * 2)
